# Let's generate the baseline

In [37]:
%run ../script/webnlg.py
import pandas as pd

pd.set_option('max_colwidth', 100)

test = WebNLGDataset("../data/webnlg2017/testdata_with_lex.xml")

test.ldf['id'] = test.ldf.eid.str.extract('(\d+)').astype('int')
test.mdf['id'] = test.mdf.eid.str.extract('(\d+)').astype('int')

/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [31]:
references = test.ldf.groupby('id').apply(lambda g: g.ltext.tolist())

In [38]:
def join_triples(g):
    
    return ' '.join(g.mtext.str.replace('[|_]', ' ').str.replace(r'\s+', ' ').tolist())

baseline = test.mdf.groupby('id').apply(join_triples)

In [34]:
references.head()

id
1    [Abilene, Texas is served by the Abilene regional airport., Abilene Regional Airport serves the ...
2    [Adolfo Suárez Madrid–Barajas Airport can be found in Madrid, Paracuellos de Jarama, San Sebasti...
3    [The runway name of Adolfo Suárez Madrid–Barajas Airport is 18L/36R., The runway name at Adolfo ...
4    [Afonso Pena International Airport ICAO Location Idenitifier is SBCT., SBCT is the ICAO location...
5    [Afonso Pena International Airport serves the city of Curitiba., Afonso Pena International Airpo...
dtype: object

In [39]:
baseline.head()

id
1                                                     Abilene Regional Airport cityServed Abilene, Texas
2    Adolfo Suárez Madrid–Barajas Airport location "Madrid, Paracuellos de Jarama, San Sebastián de l...
3                                              Adolfo Suárez Madrid–Barajas Airport runwayName "18L/36R"
4                                      Afonso Pena International Airport ICAO Location Identifier "SBCT"
5                                                  Afonso Pena International Airport cityServed Curitiba
dtype: object

# Let's preprocess texts to use as input to NLTK BLEU

In [40]:
s_multiwordexpressions = test.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = test.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

print(t.tokenize('al-taqaddum air base serves the city of fallujah .\n'.split()))

['al-taqaddum_air_base', 'serves', 'the', 'city', 'of', 'fallujah', '.']


In [42]:
from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True)
    
    return t.tokenize(ref.split())

In [45]:
preprocessed_references = references.apply(lambda l: [preprocess(s) for s in l])
preprocessed_baseline = baseline.apply(preprocess)

# Let's calculate BLEU

In [46]:
from nltk.translate.bleu_score import corpus_bleu

In [47]:
corpus_bleu(preprocessed_references, preprocessed_baseline)

0.02525649456272137

# What's the value for one of the submissions?

In [48]:
with open('../data/webnlg2017/submissions/melbourne/final_result.txt') as f:
    
    melbourn = f.readlines()

In [49]:
preprocessed_melbourn = [preprocess(l) for l in melbourn]

In [50]:
corpus_bleu(preprocessed_references, preprocessed_melbourn)

0.238959236874247